In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support as score
import itertools
from sklearn.preprocessing import OneHotEncoder #pour le recodage des variables catégorielles  
from sklearn.preprocessing import MinMaxScaler # pour effectuer la normalisations min-max
from sklearn.compose import make_column_transformer # Applique des transformateurs aux colonnes d'un tableau
from sklearn.pipeline import make_pipeline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<H3>Load data from fichier excel</H3>

In [5]:
df=pd.read_excel('Stats multi 3 IA .xlsx', index_col=0)

<H4>Delete old row because it is null</H4>

In [6]:
df = df.iloc[:-1 , :]

<H3>change type of index</H3>

In [7]:
df.index=df.index.astype(int)

In [8]:
df

,DATE ETT,DATE NAISSANCE JJ/MM/AA,Age à l'inclusion,SEXE,SURFACE CORP (m²),GENETIQUE,MUTATION 0 - 1,MYBPC3,MYH7,TNNT2,...,CHOC DAI,TRANSPLANTATION ou LISTE DE TRANSPLANTATION,DCD,CJP,Unnamed: 53,ESC score,RECO ESC (4%),RECO ESC 6%,RECO AHA,RECO AHA étendue
N°Patient,,,,,,,,,,,,,,,,,,,,,
1,2013-06-19,1955-03-14,58.0,0.femme,1.89,3.non_fait,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,0.92,0.non,0.non,0.non,0.non
2,2015-02-11,1971-12-31,43.0,0.femme,1.61,0.negative,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,1.17,0.non,0.non,0.non,0.non
3,2012-12-19,1953-01-28,59.0,0.femme,NaN,3.non_fait,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,1.28,0.non,0.non,0.non,0.non
4,2016-01-06,1975-06-16,40.0,0.femme,1.72,0.negative,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,4.11,1.oui,0.non,1.oui,1.oui
5,2014-05-07,1988-11-19,25.0,1.homme,2.05,0.negative,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,1.41,0.non,0.non,0.non,0.non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2012-03-06,1949-01-17,63.0,0.femme,1.53,1.positive,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.oui,1.oui
428,2010-04-08,1960-10-20,49.0,1.homme,1.99,0.negative,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.oui,1.oui
429,2012-10-31,1968-03-04,44.0,0.femme,1.80,0.negative,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2.26,0.non,0.non,0.non,1.oui


In [9]:
df=df.drop(['Unnamed: 53'], axis=1)

In [10]:
df.dtypes

DATE ETT                                                                      datetime64[ns]
DATE NAISSANCE JJ/MM/AA                                                       datetime64[ns]
Age à l'inclusion                                                                    float64
SEXE                                                                                  object
SURFACE CORP (m²)                                                                    float64
GENETIQUE                                                                             object
MUTATION         0 - 1                                                               float64
MYBPC3                                                                               float64
MYH7                                                                                 float64
TNNT2                                                                                float64
TNNI3                                                                 

In [11]:
df.isnull().sum()

DATE ETT                                                                        0
DATE NAISSANCE JJ/MM/AA                                                         0
Age à l'inclusion                                                               0
SEXE                                                                            0
SURFACE CORP (m²)                                                              26
GENETIQUE                                                                       0
MUTATION         0 - 1                                                         80
MYBPC3                                                                         80
MYH7                                                                           80
TNNT2                                                                          80
TNNI3                                                                          80
GRADIENT MAX REPOS INTRA VG                                                     1
CMH O           

In [12]:
null_liste=df.columns[df.isna().any()].tolist()
len(null_liste)

37

In [13]:
null_liste.remove("DATE DERNIIERE NOUVELLES")
null_liste.remove('FIBROSE extensive')
null_liste.remove('RECO ESC (4%)')
null_liste.remove('RECO ESC 6%')      

In [14]:
null_liste

['SURFACE CORP (m²)',
 'MUTATION         0 - 1',
 'MYBPC3',
 'MYH7',
 'TNNT2',
 'TNNI3',
 'GRADIENT MAX REPOS INTRA VG',
 'ATCD Familiaux de CMH',
 'ANEVRYSME',
 'EPAISSEUR MAX ETT',
 'SIV mm',
 'Simpson biplan (%)',
 'STRAIN GLS Global',
 'd OG mm',
 'Vol OG ml',
 'PALS 4C',
 'TPLS 4C',
 'PACS 4C',
 'TPCS 4C',
 'LASRs 4C',
 'LASRe 4C',
 'LASRa 4C',
 'PALS 2C',
 'TPLS 2C',
 'PACS 2C',
 'Moy PALS',
 'Moy PACS ',
 'TPCS 2C',
 'LASRs 2C',
 'LASRe 2C',
 'LASRa 2C',
 'FEVG',
 'ESC score']

In [15]:
len(null_liste)

33

In [16]:
for i in null_liste:
    if df[i].value_counts().sort_index().index[0]==0.0 and df[i].value_counts().sort_index().index[1]==1.0 and len(df[i].value_counts().index)==2:
        df[i].fillna(value=df[i].value_counts().idxmax(), inplace=True)
        
    else:
        moy=df[i].mean()
        moy="{:.2f}".format(moy)
        df[i].fillna(value=moy, inplace=True)

In [17]:
boolcol=[ 'MUTATION         0 - 1', 'MYBPC3','MYH7', 'TNNT2', 'TNNI3', 'CMH O ',
'ATCD Familiaux de CMH', 'ATCD familliaux de MS', 'TVNS', 'syncope',
'ANEVRYSME', 'STRAIN OG','IRM Cardiaque                                                             ',
'TV', 'DAI', 'CHOC DAI', 'TRANSPLANTATION ou LISTE DE TRANSPLANTATION', 'DCD','CJP']
df[boolcol]=df[boolcol].astype(int)

In [18]:
df.isna().sum()

DATE ETT                                                                        0
DATE NAISSANCE JJ/MM/AA                                                         0
Age à l'inclusion                                                               0
SEXE                                                                            0
SURFACE CORP (m²)                                                               0
GENETIQUE                                                                       0
MUTATION         0 - 1                                                          0
MYBPC3                                                                          0
MYH7                                                                            0
TNNT2                                                                           0
TNNI3                                                                           0
GRADIENT MAX REPOS INTRA VG                                                     0
CMH O           

In [19]:
df['FIBROSE extensive'].fillna(value=df['FIBROSE extensive'].value_counts().idxmax(), inplace=True)

In [20]:
df['RECO ESC (4%)'].fillna(value=df['RECO ESC (4%)'].value_counts().idxmax(), inplace=True)
df['RECO ESC 6%'].fillna(value=df['RECO ESC 6%'].value_counts().idxmax(), inplace=True)

In [21]:
df.isna().sum()

DATE ETT                                                                       0
DATE NAISSANCE JJ/MM/AA                                                        0
Age à l'inclusion                                                              0
SEXE                                                                           0
SURFACE CORP (m²)                                                              0
GENETIQUE                                                                      0
MUTATION         0 - 1                                                         0
MYBPC3                                                                         0
MYH7                                                                           0
TNNT2                                                                          0
TNNI3                                                                          0
GRADIENT MAX REPOS INTRA VG                                                    0
CMH O                       

In [22]:
for i in range(len(df['DATE DERNIIERE NOUVELLES'])):
    if pd.isna(df['DATE DERNIIERE NOUVELLES'].iloc[i]):
        df['DATE DERNIIERE NOUVELLES'].iloc[i]=df['DATE DERNIIERE NOUVELLES'].iloc[i-1]

C:\Users\hacen\AppData\Local\Temp\ipykernel_6080\1786708746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE DERNIIERE NOUVELLES'].iloc[i]=df['DATE DERNIIERE NOUVELLES'].iloc[i-1]


In [23]:
df.isna().sum()

DATE ETT                                                                      0
DATE NAISSANCE JJ/MM/AA                                                       0
Age à l'inclusion                                                             0
SEXE                                                                          0
SURFACE CORP (m²)                                                             0
GENETIQUE                                                                     0
MUTATION         0 - 1                                                        0
MYBPC3                                                                        0
MYH7                                                                          0
TNNT2                                                                         0
TNNI3                                                                         0
GRADIENT MAX REPOS INTRA VG                                                   0
CMH O                                   

In [24]:
df['TV'].value_counts()

0    402
1     29
Name: TV, dtype: int64

In [25]:
def f(x,y):
    return x+y/2
df["LASRa 2C"] = pd.to_numeric(df["LASRa 2C"], downcast="float")
df["LASRs 2C"] = pd.to_numeric(df["LASRs 2C"], downcast="float")
df['MOY_LASR'] = df[['LASRa 2C','LASRs 2C']].apply(lambda x: f(*x), axis=1) 

In [26]:
df['MOY_LASR']

N°Patient
1     -0.120
2     -0.875
3     -0.820
4     -0.315
5     -0.825
       ...  
427   -0.120
428   -0.120
429   -0.120
430   -0.120
431   -0.120
Name: MOY_LASR, Length: 431, dtype: float64

In [27]:
# conversion en type datetime
df['DATE ETT'] = pd.to_datetime(df['DATE ETT'])
df['DATE NAISSANCE JJ/MM/AA'] = pd.to_datetime(df['DATE NAISSANCE JJ/MM/AA'])
df['DATE DERNIIERE NOUVELLES'] = pd.to_datetime(df['DATE DERNIIERE NOUVELLES'])



# normalisation des dates en entiers
df['DATE ETT'] = (df['DATE ETT'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('60s')
df['DATE NAISSANCE JJ/MM/AA'] = (df['DATE NAISSANCE JJ/MM/AA'] - pd.Timestamp("1920-01-01")) // pd.Timedelta('60s')
df['DATE DERNIIERE NOUVELLES'] = (df['DATE DERNIIERE NOUVELLES'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('60s')

df['DATE ETT'] = df['DATE ETT'].astype(int)
df['DATE NAISSANCE JJ/MM/AA'] = df['DATE NAISSANCE JJ/MM/AA'].astype(int)
df['DATE DERNIIERE NOUVELLES'] = df['DATE DERNIIERE NOUVELLES'].astype(int)



In [28]:
Y = df['DCD']
X = df.drop('DCD',axis=1)
numeric = ["Age à l'inclusion","SURFACE CORP (m²)","GRADIENT MAX REPOS INTRA VG","EPAISSEUR MAX ETT","SIV mm","Simpson biplan (%)","STRAIN GLS Global","d OG mm","Vol OG ml","Vol OG ml",
"TPLS 4C","PACS 4C","TPCS 4C","LASRs 4C","LASRe 4C","LASRa 4C","PALS 2C","TPLS 2C","PACS 2C","Moy PALS","Moy PACS ","TPCS 2C",
"LASRs 2C","LASRe 2C","LASRa 2C","FEVG","suivi"] #sélection des variables à normaliser
scaler = MinMaxScaler()
X[numeric] = scaler.fit_transform(X[numeric])
X.head()

,DATE ETT,DATE NAISSANCE JJ/MM/AA,Age à l'inclusion,SEXE,SURFACE CORP (m²),GENETIQUE,MUTATION 0 - 1,MYBPC3,MYH7,TNNT2,...,DAI,CHOC DAI,TRANSPLANTATION ou LISTE DE TRANSPLANTATION,CJP,ESC score,RECO ESC (4%),RECO ESC 6%,RECO AHA,RECO AHA étendue,MOY_LASR
N°Patient,,,,,,,,,,,,,,,,,,,,,
1,22860000,18512640,0.540541,0.femme,0.474212,3.non_fait,1,0,0,0,...,0,0,0,0,0.92,0.non,0.non,0.non,0.non,-0.120
2,23726880,27348480,0.337838,0.femme,0.223685,0.negative,0,0,0,0,...,0,0,0,0,1.17,0.non,0.non,0.non,0.non,-0.875
3,22597920,17396640,0.554054,0.femme,0.474212,3.non_fait,1,0,0,0,...,0,0,0,0,1.28,0.non,0.non,0.non,0.non,-0.820
4,24200640,29167200,0.297297,0.femme,0.322106,0.negative,0,0,0,0,...,0,0,0,0,4.11,1.oui,0.non,1.oui,1.oui,-0.315
5,23323680,36230400,0.094595,1.homme,0.617370,0.negative,0,0,0,0,...,0,0,0,0,1.41,0.non,0.non,0.non,0.non,-0.825


In [29]:
categoriel = ['SEXE','GENETIQUE','FIBROSE extensive','RECO ESC (4%)','RECO ESC 6%','RECO AHA','RECO AHA étendue ']
X= pd.concat([X,pd.get_dummies(X[categoriel])],axis=1)
X.drop(categoriel,axis=1, inplace=True)


In [30]:
X.head()

,DATE ETT,DATE NAISSANCE JJ/MM/AA,Age à l'inclusion,SURFACE CORP (m²),MUTATION 0 - 1,MYBPC3,MYH7,TNNT2,TNNI3,GRADIENT MAX REPOS INTRA VG,...,FIBROSE extensive_1.oui,"RECO ESC (4%)_0,non",RECO ESC (4%)_0.non,RECO ESC (4%)_1.oui,RECO ESC 6%_0.non,RECO ESC 6%_1.oui,RECO AHA_0.non,RECO AHA_1.oui,RECO AHA étendue _0.non,RECO AHA étendue _1.oui
N°Patient,,,,,,,,,,,,,,,,,,,,,
1,22860000,18512640,0.540541,0.474212,1,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0
2,23726880,27348480,0.337838,0.223685,0,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0
3,22597920,17396640,0.554054,0.474212,1,0,0,0,0,0.072464,...,0,0,1,0,1,0,1,0,1,0
4,24200640,29167200,0.297297,0.322106,0,0,0,0,0,0.000000,...,1,0,0,1,1,0,0,1,0,1
5,23323680,36230400,0.094595,0.617370,0,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0


In [31]:
#renommer les colonnes (supprission des espaces....)
X.columns = X.columns.str.replace(" ", "")
X = X.rename(columns={'DATEETT': 'DETT', 'DATENAISSANCEJJ/MM/AA': 'DNAIS',"Ageàl'inclusion":'AGEINC'
                             ,"SURFACECORP(m²)":"SURFC","MUTATION0-1":"MUT01","CHOCDAI":"CHOC","TRANSPLANTATIONouLISTEDETRANSPLANTATION":"TRSPL"
                             ,"FIBROSEextensive":"FIBR", "IRMCardiaque":"IRMC",'VolOGml':"VOG","dOGmm":"DOG","STRAINGLSGlobal":"GLSVG"
                             ,"EPAISSEURMAXETT":"EPMAXETT","ANEVRYSME":"ANVR","syncope":"SYNC","ATCDfamilliauxdeMS":"ANTEMS","ATCDFamiliauxdeCMH":"ANTECMH0"
                             ,"GRADIENT MAX REPOS INTRA VG":"GRDMAXVG"})


X.head()

,DETT,DNAIS,AGEINC,SURFC,MUT01,MYBPC3,MYH7,TNNT2,TNNI3,GRADIENTMAXREPOSINTRAVG,...,FIBROSEextensive_1.oui,"RECOESC(4%)_0,non",RECOESC(4%)_0.non,RECOESC(4%)_1.oui,RECOESC6%_0.non,RECOESC6%_1.oui,RECOAHA_0.non,RECOAHA_1.oui,RECOAHAétendue_0.non,RECOAHAétendue_1.oui
N°Patient,,,,,,,,,,,,,,,,,,,,,
1,22860000,18512640,0.540541,0.474212,1,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0
2,23726880,27348480,0.337838,0.223685,0,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0
3,22597920,17396640,0.554054,0.474212,1,0,0,0,0,0.072464,...,0,0,1,0,1,0,1,0,1,0
4,24200640,29167200,0.297297,0.322106,0,0,0,0,0,0.000000,...,1,0,0,1,1,0,0,1,0,1
5,23323680,36230400,0.094595,0.617370,0,0,0,0,0,0.000000,...,0,0,1,0,1,0,1,0,1,0


In [32]:
X['DCD'] = Y

<h3> ma dataclean est dans X </h3>

In [33]:
X.to_csv('X.csv', index=False)